# Disecting Dynamic Programming (DP)

We pick one very easy dynamic programming problem named `0/1 Knapsack` Problem. The problem is apparently very simple. In all the resources book or online, people pick this problem for teaching DP. Where they proceed as follows:
- Recursive Solution
- Top Down approach with memoization
- Bottom Up approach with Tabulation

Before we move to the top-down and bottom-up approach, I want to spend some time on the first recursive solution. 

let's see what's the **logical flow** for solving this problem.

![image](images/01_knapsack.png)
[image_courtsey](https://www.educative.io/collection/page/5668639101419520/5633779737559040/5666387129270272)

**TL;DR**: At each decision node, 2 branches are there. The recursive code actually programs the behaviour of each decision node.

At each node 2 decision flow

```python
# DECESIION 1: select the current element
if weight.current_element <= capacity:
    include current element
elif weight.current_element > capacity:
    exclude current element

# DECESIION 2: skip the current element
exclude the current element
```

**Important point**: `both of these 2 decision branches have to be taken`. So while writing the `if-else` statement, one must be very careful.


Below there are 3 resursive implementations
- wrong implementation
- correct implementation but not optimum
- correct implementation but optimum

In [1]:
from typing import List

In [14]:
def knapsack_recur_wrong_1(weight, profit, capacity, idx):
    if idx < 0 or idx >= len(profit):
        return 0
    
    prft1 = 0
    if weight[idx] <= capacity:
        prft1 = profit[idx] + knapsack_recur_wrong_1(weight, profit, capacity - weight[idx], idx+1)
    
    prft2 = 0
    if weight[idx] > capacity:
        prft2 = knapsack_recur_wrong_1(weight, profit, capacity, idx+1)
        
    return max([prft1, prft2])

def knapsack_recur_wrong_2(weight, profit, capacity, idx):
    if idx < 0 or idx >= len(profit):
        return 0
    
    if capacity < 0:
        return 0
    
    # include
    prft1 = profit[idx] + knapsack_recur_wrong_2(weight, profit, capacity - weight[idx], idx+1)
    
    # exclude
    prft2 = knapsack_recur_wrong_2(weight, profit, capacity, idx+1)
        
    return max([prft1, prft2])

def knapsack_recur_correct_optm(weight, profit, capacity, idx):
    if idx < 0 or idx >= len(profit):
        return 0
    
    prft1 = 0
    if weight[idx] <= capacity:
        prft1 = profit[idx] + knapsack_recur_correct_optm(weight, profit, capacity - weight[idx], idx+1)
    
    prft2 = 0
    prft2 = knapsack_recur_correct_optm(weight, profit, capacity, idx+1)
        
    return max([prft1, prft2])

def knapsack_recur_correct_notoptm(weight, profit, capacity, idx):
    if idx < 0 or idx >= len(profit):
        return 0
    
    # exclude the current item at idx
    prft2 = 0
    prft2 = knapsack_recur_correct_notoptm(weight, profit, capacity, idx+1)
    
    # considering the current item
    #    if weight < capacity include else exclude 
    prft1 = 0
    if weight[idx] <= capacity:
        prft1 = profit[idx] + knapsack_recur_correct_notoptm(weight, profit, capacity - weight[idx], idx+1)
    else:
        prft1 = knapsack_recur_correct_notoptm(weight, profit, capacity, idx+1)
    
    return max([prft1, prft2])

In [2]:
ls_profit = [1, 6, 10, 16]
ls_weight = [1, 2, 3, 5]

ls_capacity = [5,7,9,10]

for i in ls_capacity:
    capacity = i
    print('-'*40+'| capacity: {} |'.format(capacity)+'-'*40)
    print('wrong 1 output: {}'.format(knapsack_recur_wrong_1(ls_weight, ls_profit, capacity, 0)))
    print('wrong 2 output: {}'.format(knapsack_recur_wrong_2(ls_weight, ls_profit, capacity, 0)))
    print('correct not-optimum output: {}'.format(knapsack_recur_correct_notoptm(ls_weight, ls_profit, capacity, 0)))
    print('correct optimum output: {}'.format(knapsack_recur_correct_optm(ls_weight, ls_profit, capacity, 0)))

----------------------------------------| capacity: 5 |----------------------------------------
wrong 1 output: 7
wrong 2 output: 32
correct not-optimum output: 16
correct optimum output: 16
----------------------------------------| capacity: 7 |----------------------------------------
wrong 1 output: 17
wrong 2 output: 33
correct not-optimum output: 22
correct optimum output: 22
----------------------------------------| capacity: 9 |----------------------------------------
wrong 1 output: 17
wrong 2 output: 33
correct not-optimum output: 27
correct optimum output: 27
----------------------------------------| capacity: 10 |----------------------------------------
wrong 1 output: 17
wrong 2 output: 33
correct not-optimum output: 32
correct optimum output: 32


**Here are my questions:**
- Q1. Where is the problem in `knapsack_recur_wrong_1()`?
- Q2. Where is the problem in `knapsack_recur_wrong_2()`?
- Q3. What is the optimization between the 2 correct implementation?

------

Now let's jump to the question answering session

**Where is the problem in** `knapsack_recur_wrong_1()`?

The 1st wrong implementation is actually not taking these 2 decision branches at each node. It's taking `either one of them`.

![image](images/wrong_implementation.png)

Which is wrong.

**Where is the problem in** `knapsack_recur_wrong_2()`?

It's not taking any decision, whether the current element weight is acceptable or not, while including the element. Therefore the profit of the element is always added in the inclusion scenario. Therefore, the total profit is always >= optimal profit. So wrong solution


**What is the optimization between the 2 correct implementation?**

Before anwering the question, let's see what's happening inside the yellow decision node (refer to the earlier image).

![image](images/correct_implementation_1.png)

Now instead of 2 exclude decision, only 1 is suffecient. That's the difference between optimum and notoptimum correct implementation.


**FIN**

In [17]:
def solve_knapsack(profit:List, weight:List, capacity:int):
    return solve_knapsack_util(profit, weight, capacity, 0)

def solve_knapsack_util(profit:List, weight:List, capacity:int, idx:int):
    """Returns profit
    """
    #base check
    if capacity <= 0 or idx >= len(profit): return 0
    
    #include: current index, used in profit calculation 
    profit_include = 0
    if weight[idx] <= capacity:
        profit_include = profit[idx] + solve_knapsack_util(profit, weight, capacity - weight[idx], idx+1) 
    
    # exclude: current index not used in profit calculation
    profit_exclude = solve_knapsack_util(profit, weight, capacity, idx+1) 
    
    return max([profit_include, profit_exclude])

In [29]:
%%time
print(solve_knapsack([1, 6, 10, 16], [1, 2, 3, 5], 7))
print(solve_knapsack([1, 6, 10, 16], [1, 2, 3, 5], 6))

22
17
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 260 µs


In [52]:
def solve_knapsack_TD(profit:List, weight:List, capacity:int):
    
    solution = [[-1 for i in range(capacity+1)] for j in range(len(profit))]
    result = solve_knapsack_TD_util(solution, profit, weight, capacity, 0)
    print(solution)
    return result
    
def solve_knapsack_TD_util(solution:List[List[int]], profit:List, weight:List, capacity:int, idx:int):
    
    #base check
    if capacity <= 0 or idx >= len(profit): return 0
    
    if solution[idx][capacity]!=-1: 
        print("HIT")
        return solution[idx][capacity]
    
    #include: current index, used in profit calculation 
    profit_include = 0
    if weight[idx] <= capacity:
        profit_include = profit[idx] + solve_knapsack_TD_util(solution, profit, weight, capacity - weight[idx], idx+1) 
        
    # exclude: current index not used in profit calculation
    profit_exclude = 0
    profit_exclude = solve_knapsack_TD_util(solution, profit, weight, capacity, idx+1)
    
    solution[idx][capacity] = max([profit_include, profit_exclude])
    
    return solution[idx][capacity]

def solve_knapsack_BU(profit:List, weight:List, capacity:int):
    
    # base cases:
    n = len(weight)
    if capacity <=0 or n==0 or n != len(profit): return 0
    
    # create solution matrix
    solution = [[0 for i in range(capacity+1)] for j in range(n)]
    
    # fill: profit=0 if capacity=0
    for i in range(n):
        solution[i][0] = 0
    
    # if only single element and it's weight<capacity put it's profit
    for c in range(0,capacity+1):
        if weight[0] <= c:
            solution[0][c] = profit[0]
            
    for i in range(1, n):
        for c in range(1,capacity+1):
            profit1, profit2 = 0, 0
            if weight[i] <= c:
                profit1 = profit[i] + solution[i-1][c - weight[i]]
            
            profit2 = solution[i-1][c]
            
            solution[i][c] = max(profit1, profit2)
            
    return solution[n-1][capacity]

In [48]:
%%time
print(solve_knapsack_TD([1, 6, 10, 16], [1, 2, 3, 5], 7))
print(solve_knapsack_TD([1, 6, 10, 16], [1, 2, 3, 5], 6))

HIT
[[-1, -1, -1, -1, -1, -1, -1, 22], [-1, -1, -1, -1, -1, -1, 16, 22], [-1, -1, -1, -1, 10, 16, 16, 16], [-1, 0, 0, 0, 0, 16, 16, 16]]
22
HIT
[[-1, -1, -1, -1, -1, -1, 17], [-1, -1, -1, -1, -1, 16, 16], [-1, -1, -1, 10, 10, 16, 16], [-1, 0, 0, 0, 0, 16, 16]]
17
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 472 µs


In [53]:
print(solve_knapsack_BU([1, 6, 10, 16], [1, 2, 3, 5], 7))
print(solve_knapsack_BU([1, 6, 10, 16], [1, 2, 3, 5], 6))

22
17
